# CMSC421 Final Project
Team Members: Jordan Maggin, Lex Kim, Zhuo Cheng Xie, Neel Jay, Hylene Wu

Project Description: The program will train on the set of images and gather information on airplanes from these images. With this information, it will attempt to both locate and identify airplanes in different sets of videos.


Importing dataset from Roboflow Project

In [3]:
import os
root_dir = os.getcwd()
!mkdir {root_dir}\datasets
%cd {root_dir}\datasets

# paste the exported dataset code here
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="nr1RFEWRZXe3o3eQV1ns")
project = rf.workspace("cmsc421-final-project").project("vehicle-detection-ansgq")
version = project.version(1)
model = version.model
dataset = version.download("yolov8")

C:\Users\GW\Downloads\421final\datasets


A subdirectory or file C:\Users\GW\Downloads\421final\datasets already exists.


Custom Training

(Weights are saved to /content/runs/detect/train)

# Backend Functions

Could be included in a Flask API if app is ever deployed.

In [1]:
!pip install ultralytics==8.0.196 roboflow supervision --quiet

In [2]:
import cv2
import numpy as np

# from ultralytics import YOLO
from roboflow import Roboflow

import supervision as sv

In [3]:
# API_KEY = "nr1RFEWRZXe3o3eQV1ns"
API_KEY = "WwDE4oQPlOhliOUfZFkn"
roboflow_workspace = "cmsc421-final-project"
roboflow_project = "plane-detection-eyzak"
version = 2

In [4]:
def get_roboflow_model(workspace, project, version=1):
    rf = Roboflow(api_key=API_KEY)
    model = rf.workspace(workspace).project(project).version(version).model
    return model
model = get_roboflow_model(roboflow_workspace, roboflow_project, version=1)

loading Roboflow workspace...
loading Roboflow project...


In [5]:
def annotate_image(model, image_path, output_path):    
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    image = cv2.imread(image_path)
    
    pred = model.predict(image_path, confidence=0.5, overlap=0.5).json()
    detections = sv.Detections.from_inference(pred)
    
    labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]

    annotated_image = bounding_box_annotator.annotate(
        scene=image, detections=detections
    )
    
    annotated_image = label_annotator.annotate(
        annotated_image, detections=detections, labels=labels
    )
    
    cv2.imwrite(output_path, annotated_image)


In [6]:
def annotate_video(model, video_path, output_path):
    tracker = sv.ByteTrack()
    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    trace_annotator = sv.TraceAnnotator(trace_length=80, thickness=3)
    heat_map_annotator = sv.HeatMapAnnotator(radius=60, kernel_size=35)

    video_info = sv.VideoInfo.from_video_path(video_path)
    frames_generator = sv.get_video_frames_generator(video_path)
    pred = None

    with sv.VideoSink(output_path, video_info) as sink:
        for i, frame in enumerate(frames_generator):
#             print(f"Frame {i}")
            if i % 20 == 0:
                pred = model.predict(frame, confidence=0.5, overlap=0.5).json()

            """
            detections = sv.Detections(
                xyxy=pred['predictions'][0]['x'],
                confidence=pred['predictions'][0]['confidence'],
                class_id=pred['predictions'][0]['class']
            )
            """
            detections = sv.Detections.from_inference(pred)
            detections = tracker.update_with_detections(detections)
            labels = [
                f"{class_name} {confidence:.2f}"
                for class_name, confidence
                in zip(detections['class_name'], detections.confidence)
            ]

            annotated_frame = bounding_box_annotator.annotate(
                scene=frame.copy(), detections=detections
            )
            annotated_frame = label_annotator.annotate(
                scene=annotated_frame, detections=detections, labels=labels
            )
            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame, detections=detections
            )
            annotated_frame = heat_map_annotator.annotate(
                scene=annotated_frame, detections=detections
            )

            sink.write_frame(annotated_frame)

In [7]:
# # Example use
# video_path = "test_video.mov"
# output_path = "annotated_video.mp4"
# annotate_video(model, video_path, output_path)

# Simple GUI with PySimpleGUI

In [8]:
# Installing PySimpleGUI
!python -m pip install pysimplegui
!pip install pillow

In [9]:
# Imports
import PySimpleGUI as sg
import cv2
import numpy as np
from PIL import Image
import io
import os

In [ ]:
# Defines potential filetypes to search for
file_types = [("JPEG (*.jpg)", "*.jpg"),
              ("All files (*.*)", "*.*")]

def main():
    # Defines list of files from selected folder
    file_list_column_img = [
        [
            sg.Text("Image Folder"),
            sg.In(size=(25, 1), enable_events=True, key="-IMG FOLDER-"),
            sg.FolderBrowse(),
        ],
        [
            sg.Listbox(
                values=[], enable_events=True, size=(60, 20), key="-IMG LIST-"
            )
        ],
    ]
    
    file_list_column_vid = [
        [
            sg.Text("Video Folder"),
            sg.In(size=(25, 1), enable_events=True, key="-VID FOLDER-"),
            sg.FolderBrowse(),
        ],
        [
            sg.Listbox(
                values=[], enable_events=True, size=(60, 20), key="-VID LIST-"
            )
        ],
    ]
    
    # Image display
    image_display_column = [
        [
            sg.Image(key="-IMAGE-"),
        ]
    ]
    
    # Video column -- lets user know if data has been processed or not.
    vid_col = [
        [
            sg.Text("Press button once video is selected to process."),
        ],
        [
#             sg.Text(),
            sg.Text("Video not yet processed.", size=(40, 1), key="-TOUT-"),
        ],
        [
            sg.Button("Process Video"),
        ]
    ]
    
    # ----------------LAYOUTS
    # Defines layout for the video processor
    
    layout_vid = [
        [
            sg.Column(file_list_column_vid, element_justification='c'),
            sg.VSeparator(),
            sg.Column(vid_col, element_justification='c')
        ],
    ]
    
    # Defines layout for image viewer
    layout_img = [
        #[sg.Image(key="-IMAGE-")],
        [
            sg.Column(file_list_column_img, element_justification='c'),
            sg.VSeparator(),
            sg.Column(image_display_column, element_justification='c'),
        ],
        
    ]
    
    layout = [
        [
            sg.Column(layout_img, key='-COL1-'), sg.Column(layout_vid, visible=False, key='-COL2-')
        ],
        [
            sg.Button("Swap Program")
        ],
        [
            sg.Button("Exit Program")
        ],
    ]
    
    
    # Applies the layout to the window
    window = sg.Window("CMSC421 Computer Vision // Final Project", layout, element_justification='c')
    layout = 1
    
    ###########
    # UPDATER #
    ###########
    # Updates the window with corresponding values and images
    while True:
        event, values = window.read()
        
        # Closes the window upon pressing the Exit button or closing the window.
        if event == "Exit Program" or event == sg.WIN_CLOSED:
            break
        # Swaps program
        if event == "Swap Program":
            window[f'-COL{layout}-'].update(visible=False)
            if layout == 1:
                layout = 2
            else:
                layout = 1
            window[f'-COL{layout}-'].update(visible=True)
                    
        # Events for IMG
        if event == "-IMG FOLDER-":
            folder = values["-IMG FOLDER-"]
            try:
                # Get list of files in folder
                file_list = os.listdir(folder)
            except:
                file_list = []

            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".jpg"))
            ]
            window["-IMG LIST-"].update(fnames)
        if event == "-IMG LIST-":  # A file was chosen from the listbox
            try:
                filename_input = os.path.join(
                    values["-IMG FOLDER-"], values["-IMG LIST-"][0]
                )
                
                filename_output = os.path.join(
                    values["-IMG FOLDER-"], 'prediction.jpg'
                )

                annotate_image(model,filename_input,filename_output)
                image = Image.open(filename_output)
                image.thumbnail((400, 400))
                bio = io.BytesIO()
                image.save(bio, format="PNG")
                window["-IMAGE-"].update(data=bio.getvalue())
            except:
                pass
            
        # Events for VID
        if event == "-VID FOLDER-":
            folder = values["-VID FOLDER-"]
            try:
                # Get list of files in folder
                file_list = os.listdir(folder)
            except:
                file_list = []

            fnames = [
                f
                for f in file_list
                if os.path.isfile(os.path.join(folder, f))
                and f.lower().endswith((".mov"))
            ]
            window["-VID LIST-"].update(fnames)
        if event == "-VID LIST-":  # A file was chosen from the listbox
            try:
                pass
#                 filename_input = os.path.join(
#                     values["-VID FOLDER-"], values["-VID LIST-"][0]
#                 )
                
#                 filename_output = os.path.join(
#                     values["-VID FOLDER-"], 'prediction.mp4'
#                 )
#                 window["-TOUT-"].update("Video processed.")
                
#                 annotate_video(model,filename_input,filename_output)
            except:
                pass
        if event == "Process Video":
            try:
                pass
                filename_input = os.path.join(
                    values["-VID FOLDER-"], values["-VID LIST-"][0]
                )
                
                filename_output = os.path.join(
                    values["-VID FOLDER-"], 'prediction.mp4'
                )
                window["-TOUT-"].update("Video processed.")
                
                annotate_video(model,filename_input,filename_output)
            except:
                pass
#         if event == "Load Image":  # NOTE: This part is currently unused, but is available for debugging.
#             filename = values["-FILE-"]
#             if os.path.exists(filename):
#                 # NOTE: If you want to do predictions, they would have to be placed here, prior to conversion.
#                 image = Image.open(values["-FILE-"])
#                 image.thumbnail((400, 400))
#                 bio = io.BytesIO()
#                 image.save(bio, format="PNG")
#                 window["-IMAGE-"].update(data=bio.getvalue())
    
    # Closes window after break
    window.close()

main()

C:/Users/GW/Downloads/421final/datasets/Vehicle-Detection-1/valid/images\prediction.jpg
